In [1]:
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_fixed
from PIL import Image
import base64
from io import BytesIO

import PIL.Image
client = OpenAI(
    base_url='https://api.openai-proxy.org/v1',
    api_key='Your Keys',
)

In [53]:
def encode_image(image):
    # Check if the image is a string path and open it
    if isinstance(image, str):
        image = Image.open(image)   
        
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_bytes = buffered.getvalue()
    img_base64 = base64.b64encode(img_bytes)
    img_str = img_base64.decode('utf-8')
    
    return img_str

def encode_image_gpt4v(image):
    return 'data:image/jpeg;base64,' + encode_image(image)

In [73]:
def get_result(img_path):
    question='{"Task": "Please provide me with the quantity, position, and relationship of all different objects in the given image in JSON format.\
    For quantity, count all the quantities of different objects. If it is 0, do not output that class,\
    For the positions of different objects, the position of each individual unit in each type of object needs to be output(the positions are divided into nine grids, namely the top left corner),\
    top,top right corner,left,center,right,left lower corner,lower,right lower corner)。\
    For relationships, output the relationships between all objects in the image in the form of triplets, [object2, relation,object2],\
    Please refer to the following JSON format for specific formatting, and use this JSON format for output (just output one JSON)",\
    "Restriction": "Provide JSON response in a format stricter than<format> without wrapping it in code block markers"\
    "format":{"number":{"man":3,"car":2,"desk":1,"woman":3},"location":{"man":{1:"center",2:"top"},"car":{1,:"top right corner",2:left} },"relationship":[["man","near to","car"],["car","on","load"],["woman","has","car"]]} }'


    
    img_url=encode_image_gpt4v(img_path)
    
    messages=[{
        'role':'user',
        'content':[
            {
                "text":question,
                "type": "text"
            },
            {
                "type": "image_url",
                "image_url":{"url":img_url}
            }
        ]
    }]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-4o-2024-08-06",
    )

    return chat_completion

In [51]:
import os
import pickle
dir_list=os.listdir('JPEGImages')

for i in dir_list[-63:]:
    result=get_result(os.path.join('JPEGImages',i)).choices[0].message.content

    with open(os.path.join('gpt_answer',i[:-4]+'.pickle'),'wb') as f:
        pickle.dump(result,f)

    print(i)

507.jpg
508.jpg
509.jpg
510.jpg
512.jpg
513.jpg
514.jpg
515.jpg
517.jpg
518.jpg
521.jpg
524.jpg
525.jpg
526.jpg
529.jpg
530.jpg
531.jpg
532.jpg
533.jpg
535.jpg
536.jpg
537.jpg
538.jpg
540.jpg
543.jpg
544.jpg
545.jpg
549.jpg
551.jpg
554.jpg
556.jpg
558.jpg
559.jpg
560.jpg
561.jpg
563.jpg
564.jpg
565.jpg
566.jpg
567.jpg
568.jpg
569.jpg
570.jpg
572.jpg
573.jpg
574.jpg
576.jpg
579.jpg
580.jpg
582.jpg
583.jpg
584.jpg
585.jpg
586.jpg
587.jpg
588.jpg
590.jpg
591.jpg
592.jpg
593.jpg
594.jpg
596.jpg
598.jpg
